In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [2]:
# Load data from JSON file
file_path = 'C:/Users/HP/Desktop/angles_output_3.json'
data = []
with open(file_path) as f:
    for line in f:
        data.append(json.loads(line))

In [3]:
# Convert JSON data to DataFrame
df = pd.DataFrame(data)

In [4]:
# Separate features and labels
X = df.drop('Label', axis=1).values
y = df['Label'].values

In [5]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [7]:
# Reshape data for LSTM
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [8]:
# Build LSTM model
lstm_model = Sequential()
lstm_model.add(Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(LSTM(50))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [9]:
# Compile model
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Train model
lstm_history = lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_data=(X_test_lstm, y_test))

Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3546 - loss: 1.5538 - val_accuracy: 0.6667 - val_loss: 1.2972
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6030 - loss: 1.2497 - val_accuracy: 0.7196 - val_loss: 1.1567
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7126 - loss: 1.1261 - val_accuracy: 0.8095 - val_loss: 1.0377
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7825 - loss: 1.0092 - val_accuracy: 0.8201 - val_loss: 0.9295
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7921 - loss: 0.9183 - val_accuracy: 0.8201 - val_loss: 0.8591
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8310 - loss: 0.8413 - val_accuracy: 0.8307 - val_loss: 0.8153
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8551 - loss: 0.7722 - val_accuracy: 0.8730 - val_loss: 0.7650
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8341 - loss: 0.7725 - val_accuracy: 0.8942 - val_loss

In [11]:
# Save the training history
with open('training_history.json', 'w') as f:
    json.dump(lstm_history.history, f)

In [12]:
# Save LSTM model
lstm_model.save('verification_model_implementation_2.h5')

In [1]:
import json
import numpy as np
import math
import cv2
import mediapipe as mp
from tensorflow.keras.models import load_model

In [2]:
# Define angle thresholds for each pose
ANGLE_THRESHOLDS = {
    'downdog': {
        'left_elbow_angle': (150, 200),
        'right_elbow_angle': (150, 200),
        'left_shoulder_angle': (150, 220),
        'right_shoulder_angle': (150, 220),
        'left_knee_angle': (140, 200),
        'right_knee_angle': (140, 200),
        'left_hip_angle': (60, 110),
        'right_hip_angle': (60, 110)
    },
    'goddess': {
        'left_elbow_angle': (70, 120),
        'right_elbow_angle': (70, 120),
        'left_shoulder_angle': (70, 110),
        'right_shoulder_angle': (70, 110),
        'left_knee_angle': (90, 140),
        'right_knee_angle': (90, 140),
        'left_hip_angle': (40, 90),
        'right_hip_angle': (40, 90)
    },
    'plank': {
        'left_elbow_angle': (160, 180),
        'right_elbow_angle': (160, 180),
        'left_shoulder_angle': (70, 120),
        'right_shoulder_angle': (70, 120),
        'left_knee_angle': (160, 180),
        'right_knee_angle': (160, 180),
        'left_hip_angle': (150, 180),
        'right_hip_angle': (150, 180)
    },
    'warrior2': {
        'left_elbow_angle': (160, 180),
        'right_elbow_angle': (160, 180),
        'left_shoulder_angle': (80, 120),
        'right_shoulder_angle': (80, 120),
        'left_knee_angle': (80, 120),
        'right_knee_angle': (80, 120),
        'left_hip_angle': (80, 120),
        'right_hip_angle': (80, 120)
    },
    'tree': {
        'left_elbow_angle': (160, 180),
        'right_elbow_angle': (160, 180),
        'left_shoulder_angle': (70, 120),
        'right_shoulder_angle': (70, 120),
        'left_knee_angle': (150, 180),
        'right_knee_angle': (150, 180),
        'left_hip_angle': (80, 120),
        'right_hip_angle': (80, 120)
    }
}

In [3]:
# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    angle = math.degrees(math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0]))
    angle = abs(angle)
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [4]:
# Function to extract joint coordinates from the image and calculate angles
def extract_angles_from_image(image_path, return_landmarks=False):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()
    
    # Load and process image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        
        # Extract coordinates for key joints
        def get_coords(landmark):
            return [landmark.x, landmark.y]

        left_shoulder = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER])
        right_shoulder = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER])
        left_elbow = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW])
        right_elbow = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW])
        left_wrist = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_WRIST])
        right_wrist = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST])
        left_hip = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_HIP])
        right_hip = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_HIP])
        left_knee = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_KNEE])
        right_knee = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE])
        left_ankle = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE])
        right_ankle = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE])

        # Calculate angles
        angles = {
            'left_elbow_angle': calculate_angle(left_shoulder, left_elbow, left_wrist),
            'right_elbow_angle': calculate_angle(right_shoulder, right_elbow, right_wrist),
            'left_shoulder_angle': calculate_angle(left_elbow, left_shoulder, left_hip),
            'right_shoulder_angle': calculate_angle(right_elbow, right_shoulder, right_hip),
            'left_knee_angle': calculate_angle(left_hip, left_knee, left_ankle),
            'right_knee_angle': calculate_angle(right_hip, right_knee, right_ankle),
            'left_hip_angle': calculate_angle(left_shoulder, left_hip, left_knee),
            'right_hip_angle': calculate_angle(right_shoulder, right_hip, right_knee)
        }

        if return_landmarks:
            landmarks_dict = {
                'left_elbow_angle': (left_shoulder, left_elbow, left_wrist),
                'right_elbow_angle': (right_shoulder, right_elbow, right_wrist),
                'left_shoulder_angle': (left_elbow, left_shoulder, left_hip),
                'right_shoulder_angle': (right_elbow, right_shoulder, right_hip),
                'left_knee_angle': (left_hip, left_knee, left_ankle),
                'right_knee_angle': (right_hip, right_knee, right_ankle),
                'left_hip_angle': (left_shoulder, left_hip, left_knee),
                'right_hip_angle': (right_shoulder, right_hip, right_knee)
            }
            return angles, landmarks_dict
        else:
            return angles
    else:
        return None

In [5]:
# Load LSTM model
model = load_model('verification_model_implementation_2.h5')

In [6]:
# Function to predict pose correctness with a pose label and highlight incorrect angles
def predict_pose_correctness(image_path, pose_label):
    # Load image and extract angles with landmarks
    image = cv2.imread(image_path)
    angles, landmarks = extract_angles_from_image(image_path, return_landmarks=True)
    
    if angles is None:
        return f"No pose landmarks detected for {pose_label}."

    # Reshape angles for LSTM model
    angles_array = np.array(list(angles.values())).reshape(1, 1, len(angles))

    # Predict correctness using the model
    prediction = model.predict(angles_array)
    correctness = prediction[0][0] > 0.5  # Binary classification threshold

    if correctness:
        return "yes"
    else:
        incorrect_parts = []
        incorrect_coords = []  # Store coordinates of incorrect angles

        # Check each angle against thresholds
        for angle_name, angle_value in angles.items():
            if angle_name in ANGLE_THRESHOLDS[pose_label]:  # Ensure it checks only specific pose thresholds
                min_threshold, max_threshold = ANGLE_THRESHOLDS[pose_label][angle_name]
                if not (min_threshold <= angle_value <= max_threshold):
                    incorrect_parts.append(f"{angle_name} is out of range ({angle_value:.2f}°)")
                    # Store coordinates for the incorrect angle
                    point_a, point_b, point_c = landmarks[angle_name]
                    incorrect_coords.append((point_a, point_b, point_c))

        # Draw circles around incorrect angles in the image
        for (point_a, point_b, point_c) in incorrect_coords:
            # Convert normalized coordinates to pixel coordinates
            height, width, _ = image.shape
            pa = (int(point_a[0] * width), int(point_a[1] * height))
            pb = (int(point_b[0] * width), int(point_b[1] * height))
            pc = (int(point_c[0] * width), int(point_c[1] * height))
            
            # Draw circles on each of the points involved in the incorrect angle
            cv2.circle(image, pa, 5, (0, 0, 255), -1)
            cv2.circle(image, pb, 5, (0, 0, 255), -1)
            cv2.circle(image, pc, 5, (0, 0, 255), -1)

        # Save or display the annotated image as needed
        cv2.imwrite("output_with_incorrect_angles.jpg", image)

        return "no", incorrect_parts

In [7]:
# Example usage
image_path = 'C:/Users/HP/Desktop/test_12.jfif'
pose_label = 'goddess'
correctness = predict_pose_correctness(image_path, pose_label)
print(correctness)

C:\Users\HP\anaconda3\envs\mediapipe\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
('no', ['left_elbow_angle is out of range (160.07°)', 'right_elbow_angle is out of range (168.10°)', 'left_shoulder_angle is out of range (167.87°)', 'right_shoulder_angle is out of range (171.83°)', 'left_knee_angle is out of range (177.69°)', 'right_knee_angle is out of range (171.84°)', 'left_hip_angle is out of range (168.55°)', 'right_hip_angle is out of range (92.41°)'])
